# Overview

- Notebook 555 timer-based analysis
- Notebook for [Two-probe](./two_probe.ipynb) oscillator + ADC measurements
- Notebook for [Four-probe](./four_probe.ipynb) oscillator + ADC measurements

This is 



\begin{equation}A=3\end{equation}

Okay, 








1. A big section

And then other things.